In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession \
    .builder \
    .appName("DIMs") \
    .getOrCreate()

24/08/28 01:17:49 WARN Utils: Your hostname, ahmad-HP-ZBook-17 resolves to a loopback address: 127.0.1.1; using 192.168.1.35 instead (on interface wlo1)
24/08/28 01:17:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/28 01:17:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option("header", True).option("inferSchema", True).csv("../original_dataset/Amazon Sale Report.csv").drop('index')
new_columns = [col.replace(" ", "_").replace("-", "_") for col in df.columns]
df = df.toDF(*new_columns)
df.show()


+-------------------+--------+--------------------+----------+-------------+------------------+--------+----+--------------+---+--------+------+-----------+--------------+----------------+------------+-----+------------+----+--------+
|           Order_ID|    Date|              Status|Fulfilment|ORDERS_Channel|ship_service_level|Category|Size|Courier_Status|Qty|currency|Amount|  ship_city|    ship_state|ship_postal_code|ship_country|  B2B|fulfilled_by| New|PendingS|
+-------------------+--------+--------------------+----------+-------------+------------------+--------+----+--------------+---+--------+------+-----------+--------------+----------------+------------+-----+------------+----+--------+
|405-8078784-5731545|04-30-22|           Cancelled|  Merchant|    Amazon.in|          Standard| T-shirt|   S|    On the Way|  0|     INR|647.62|     MUMBAI|   MAHARASHTRA|          400081|          IN|false|   Easy Ship|NULL|    NULL|
|171-9198151-1101146|04-30-22|Shipped - Deliver...|  Mercha

In [4]:
df.columns

['Order_ID',
 'Date',
 'Status',
 'Fulfilment',
 'ORDERS_Channel',
 'ship_service_level',
 'Category',
 'Size',
 'Courier_Status',
 'Qty',
 'currency',
 'Amount',
 'ship_city',
 'ship_state',
 'ship_postal_code',
 'ship_country',
 'B2B',
 'fulfilled_by',
 'New',
 'PendingS']

In [34]:
from pyspark.sql.functions import expr 
df= df.withColumn('Date', expr(f"to_date({'Date'}, 'mm-dd-yy')")).withColumnRenamed("date", "order_date")

In [35]:
df.createOrReplaceTempView("source_table")
source_name = "source_table"

# Lets normalize this table
['Order_ID', # fact_dim
 'Date',    # date_dim
 'Status',  # shipping_dim (SCD)
 'Fulfilment',  # shipping_dim 
 'ORDERS_Channel', # ORDERS_channel_dim
 'ship_service_level', # shipping_dim
 'Category',    #product_dim
 'Size',    #product_dim
 'Courier_Status',
 'Qty', #fact_dim
 'currency',    #currency_dim
 'Amount',  #fact_dim
 'ship_city',   #location_dim
 'ship_state',  #location_dim
 'ship_postal_code', #location_dim
 'ship_country', #location_dim
 'B2B', # fact_dim
 'fulfilled_by',    # shipping_dim 
 'New',    
 'PendingS']

# location dim test query

In [36]:
spark.sql(
    f"""
    WITH location_dim AS
    (
        SELECT
            DISTINCT 
            ship_country,
            ship_state,
            ship_city,
            ship_postal_code
        FROM
            {source_name}
    )
    SELECT
        ROW_NUMBER() OVER (ORDER BY ship_postal_code) AS location_id,
        ship_country,
        ship_state,
        ship_city,
        ship_postal_code
    FROM
        location_dim
    """
).show()

24/08/28 01:47:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:47:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:47:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:47:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+------------+----------+--------------------+----------------+
|location_id|ship_country|ship_state|           ship_city|ship_postal_code|
+-----------+------------+----------+--------------------+----------------+
|          1|        NULL|      NULL|                NULL|            NULL|
|          2|          IN|     DELHI|           NEW DELHI|          110001|
|          3|          IN|     DELHI|               Delhi|          110001|
|          4|          IN|     DELHI|           New Delhi|          110001|
|          5|          IN|     delhi|           NEW DELHI|          110001|
|          6|          IN|     Delhi|           New Delhi|          110001|
|          7|          IN|     DELHI|           NEW DELHI|          110002|
|          8|          IN|     Delhi|           New Delhi|          110002|
|          9|          IN|     DELHI|               delhi|          110002|
|         10|          IN|     DELHI|           NEW DELHI|          110003|
|         11

# date dim test query

In [42]:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

In [47]:
spark.sql(
    f"""
    WITH date_dim AS (
        SELECT
            DISTINCT 
            order_date
        FROM
            {source_name}
        WHERE
            order_date IS NOT NULL
    )
    SELECT
        ROW_NUMBER() OVER (ORDER BY order_date) AS date_id,
        order_date,
        EXTRACT(QUARTER FROM order_date) AS quarter,
        EXTRACT(MONTH FROM order_date) AS month,
        EXTRACT(WEEK FROM order_date) AS week,
        EXTRACT(DAY FROM order_date) AS day,
        EXTRACT(YEAR FROM order_date) AS year
    FROM
        date_dim
    """
).show()

24/08/28 01:51:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:51:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:51:30 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/08/28 01:51:31 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-------+----------+-------+-----+----+---+----+
|date_id|order_date|quarter|month|week|day|year|
+-------+----------+-------+-----+----+---+----+
|      1|2022-01-01|      1|    1|  52|  1|2022|
|      2|2022-01-02|      1|    1|  52|  2|2022|
|      3|2022-01-03|      1|    1|   1|  3|2022|
|      4|2022-01-04|      1|    1|   1|  4|2022|
|      5|2022-01-05|      1|    1|   1|  5|2022|
|      6|2022-01-06|      1|    1|   1|  6|2022|
|      7|2022-01-07|      1|    1|   1|  7|2022|
|      8|2022-01-08|      1|    1|   1|  8|2022|
|      9|2022-01-09|      1|    1|   1|  9|2022|
|     10|2022-01-10|      1|    1|   2| 10|2022|
|     11|2022-01-11|      1|    1|   2| 11|2022|
|     12|2022-01-12|      1|    1|   2| 12|2022|
|     13|2022-01-13|      1|    1|   2| 13|2022|
|     14|2022-01-14|      1|    1|   2| 14|2022|
|     15|2022-01-15|      1|    1|   2| 15|2022|
|     16|2022-01-16|      1|    1|   2| 16|2022|
|     17|2022-01-17|      1|    1|   3| 17|2022|
|     18|2022-01-18|

# shipping dim

In [48]:
shipping_dim_df = spark.sql(
    f"""
    SELECT
        DISTINCT
        Status,
        Fulfilment,
        ship_service_level,
        fulfilled_by
    FROM
        {source_name}
    """
)
shipping_dim_df.show()

+--------------------+----------+------------------+------------+
|              Status|Fulfilment|ship_service_level|fulfilled_by|
+--------------------+----------+------------------+------------+
|Shipped - Lost in...|  Merchant|          Standard|   Easy Ship|
|Shipped - Rejecte...|  Merchant|          Standard|   Easy Ship|
|Shipped - Returni...|  Merchant|          Standard|   Easy Ship|
|Shipped - Returne...|  Merchant|          Standard|   Easy Ship|
|Shipped - Deliver...|  Merchant|          Standard|   Easy Ship|
|Shipped - Out for...|  Merchant|          Standard|   Easy Ship|
|           Cancelled|  Merchant|          Standard|   Easy Ship|
| Shipped - Picked Up|  Merchant|          Standard|   Easy Ship|
|   Shipped - Damaged|  Merchant|          Standard|   Easy Ship|
|Pending - Waiting...|  Merchant|          Standard|   Easy Ship|
|             Pending|  Merchant|          Standard|   Easy Ship|
|             Shipped|    Amazon|         Expedited|        NULL|
|         